## ETL del dataset `steam_games`

En esta jupyter notebook se desarrolla la extracción, transformación y carga (ETL) del conjunto de datos `steam_games.json.gz`.

### Descripción de Columnas en steam_games.gz.json

| Columna        | Descripción                                  | Ejemplo                                                         |
|----------------|----------------------------------------------|-----------------------------------------------------------------|
| publisher      | Empresa publicadora del contenido             | [Ubisoft, Dovetail Games - Trains, Degica]                      |
| genres         | Género del contenido                         | [Action, Adventure, Racing, Simulation, Strategy]               |
| app_name       | Nombre del contenido                          | [Warzone, Soundtrack, Puzzle Blocks]                            |
| title          | Título del contenido                          | [The Dream Machine: Chapter 4, Fate/EXTELLA - Sweet Room Dream, Fate/EXTELLA - Charming Bunny] |
| url            | URL de publicación del contenido              | [http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/] |
| release_date   | Fecha de lanzamiento                          | [2018-01-04]                                                    |
| tags           | Etiquetas de contenido                       | [Simulation, Indie, Action, Adventure, Funny, Open World, First-Person, Sandbox, Free to Play] |
| reviews_url    | URL de reseñas del contenido                  | [http://steamcommunity.com/app/681550/reviews/?browsefilter=mostrecent&p=1] |
| specs          | Especificaciones                             | [Multi-player, Co-op, Cross-Platform Multiplayer, Downloadable Content] |
| price          | Precio del contenido                          | [4.99, 9.99, Free to Use, Free to Play]                         |
| early_access   | Acceso temprano                              | [False, True]                                                  |
| id             | Identificador único de contenido              | [761140, 643980, 670290]                                       |
| developer      | Desarrollador                                | [Kotoshiro, Secret Level SRL, Poolians.com]                    |


In [1]:
import gzip  # Para trabajar con archivos comprimidos en formato Gzip
import json  # Para trabajar con datos en formato JSON
import matplotlib.pyplot as plt  # Para crear gráficos y visualizaciones
import numpy as np  # Para operaciones matemáticas eficientes y manipulación de arrays
import pandas as pd  # Para el análisis y manipulación de datos en forma de DataFrames
import pyarrow as pa  # Herramientas para trabajar con datos en formato de flecha (Arrow)
import pyarrow.parquet as pq  # Para trabajar con archivos en formato Parquet
import seaborn as sns  # Biblioteca de visualización de datos basada en matplotlib

In [2]:
# Leemos el archivo gz que contiene el json y lo guardamos en una lista
row = []  # creamos una lista vacía para ir agregando las filas del archivo json
with gzip.open("data/steam_games.json.gz", 'rt', encoding='utf-8') as file:
    for line in file.readlines():  # leemos línea por línea
        data = json.loads(line)
        row.append(data)

# Convertimos la lista en un DataFrame
games = pd.DataFrame(row)
games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
games.shape # Obsevamos el tamaño del dataset

(120445, 13)

Ahora es necesario observar el número de nulos presentes en el dataset, para un posible imputación o eliminación de datos

In [5]:
games.isna().sum().sort_values(ascending= False)/len(games) * 100

publisher       80.004982
developer       76.058782
genres          76.045498
release_date    75.035909
title           75.021794
price           74.463033
specs           73.876043
tags            73.455104
app_name        73.321433
reviews_url     73.321433
id              73.321433
url             73.319773
early_access    73.319773
dtype: float64

Conclusión:

* Para la totalidad de columnas se presenta un valor mayor del 70% de vacíos
* No se puede concluir la eliminación de columnas ya que todos presentan alto procentaje de nulos
* Se eliminarán las filas que contengas el todas de sus registros vaciós

In [6]:
games = games.dropna(how="all").reset_index(drop=True) # Eliminamos las filas que tienen todos sus registros como nulos
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [7]:
games.shape

(32135, 13)

In [8]:
games.isna().sum().sort_values(ascending= False)/len(games) * 100

publisher       25.056792
developer       10.266065
genres          10.216275
release_date     6.432239
title            6.379337
price            4.285047
specs            2.084954
tags             0.507235
app_name         0.006224
reviews_url      0.006224
id               0.006224
url              0.000000
early_access     0.000000
dtype: float64

Conclusión:

* Se redujo bastante el número de nulos en el dataset por columna, así como su tamaño de registros, disminuyó un 70% aproximadamente
* Publisher y developer, se pueden tolerar con los vacíos
* Genres es necesario imputar datos para evitar la perdida de información

Vamos a observar las columnas para tener una mejor visión acerca de la imputación de datos

In [9]:
# se observa el tipo de dato en 'genres' para varios registros de forma aleatoria
print(games['genres'].iloc[0])
print(games['genres'].iloc[15544])
print(games['genres'].iloc[30])
print(games['genres'].iloc[14])
print(games['genres'].iloc[256])

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']
['Casual', 'Indie', 'RPG']
nan
['Casual', 'Indie', 'Simulation']
['Adventure']


In [10]:
print(games['tags'].iloc[0])
print(games['tags'].iloc[15544])
print(games['tags'].iloc[30])
print(games['tags'].iloc[14])
print(games['tags'].iloc[256])

['Strategy', 'Action', 'Indie', 'Casual', 'Simulation']
['RPG', 'Indie', 'Casual']
['Action', 'Indie', 'VR']
['Indie', 'Casual', 'Simulation']
['Adventure', 'Point & Click', 'Comedy']


In [11]:
print(games['specs'].iloc[0])
print(games['specs'].iloc[15544])
print(games['specs'].iloc[30])
print(games['specs'].iloc[14])
print(games['specs'].iloc[256])

['Single-player']
['Single-player', 'Steam Achievements', 'Steam Trading Cards']
['Single-player', 'Steam Achievements', 'Steam Leaderboards', 'HTC Vive', 'Oculus Rift', 'Tracked Motion Controllers', 'Room-Scale']
['Single-player']
['Single-player']


In [12]:
# se observa el tipo de dato en 'price'
games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [13]:
# se observa el tipo de dato en 'early_access'
games['early_access'].unique()

array([False, True], dtype=object)

In [14]:
duplicados_columnas = games[games.duplicated(subset=['id'], keep=False)] # Verificamos si hay filas duplicadas en la columna id
duplicados_columnas


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [15]:
games[games['developer']=='Rocksteady Studios,Feral Interactive (Mac)']

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",http://steamcommunity.com/app/35140/reviews/?b...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


In [16]:
# se eliminan los index deseados
id_drop = [14573, 74, 30961]
games.drop(id_drop, inplace= True)

In [17]:
duplicados_columnas = games[games.duplicated(subset=['id'], keep=False)] # Verificamos si hay filas duplicadas en la columna id
duplicados_columnas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [18]:
games['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

In [19]:
# Convertimos la columna en datetime
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')

# Rellenamos los valores nulos con Dato no disponible
games['release_date'] = games['release_date'].fillna('Dato no disponible')
games["release_date"].isnull().sum()

0

In [20]:
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
# Creamos la columna 'release_year' y usamos la propiedad .dt.year solo para las fechas válidas
games['release_year'] = games['release_date'].dt.year.where(games['release_date'].notnull(), 'Dato no disponible')

# Muestra las primeras 2 filas para verificar
games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018.0


In [21]:
games.drop('release_date', axis=1, inplace= True)

In [22]:
games['release_year'].unique()

array([2018.0, 2017.0, 'Dato no disponible', 1997.0, 1998.0, 2016.0,
       2006.0, 2005.0, 2003.0, 2007.0, 2002.0, 2000.0, 1995.0, 1996.0,
       1994.0, 2001.0, 1993.0, 2004.0, 1999.0, 2008.0, 2009.0, 1992.0,
       1989.0, 2010.0, 2011.0, 2013.0, 2012.0, 2014.0, 1983.0, 1984.0,
       2015.0, 1990.0, 1988.0, 1991.0, 1985.0, 1982.0, 1987.0, 1981.0,
       1986.0, 2021.0, 2019.0, 1975.0, 1970.0, 1980.0], dtype=object)

In [23]:
# Definimos una función para poder reemplazar los datos string de la columna "price" por flotantes 0.0

def str_a_float(valor):

    if pd.isna(valor): # si el valor es nulo retorno 0.0
        return 0.0
    try:
        flotante = float(valor) # si el valor es un flotante retorno el mismo valor
        return flotante
    except (ValueError, TypeError): # si el valor no es un flotante retorno 0.0
        return 0.0


In [24]:
games["price"] = games["price"].apply(str_a_float) # aplicamos la función a la columna "price"
games['price'].dtype

dtype('float64')

In [25]:
# SColumnas a transformar
col_to_fill = ['publisher', 'app_name', 'title', 'developer']
# Se rellenan los nulos
df_relleno = games[col_to_fill].fillna('Sin dato disponible')
# Se borran las columnas originales y se concatenan las rellenas con todo el dataframe
games = pd.concat([games.drop(col_to_fill, axis=1), df_relleno], axis=1)
games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_year,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018.0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,2018.0,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,2017.0,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017.0,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Dato no disponible,Sin dato disponible,Log Challenge,Sin dato disponible,Sin dato disponible


In [26]:
# Usar explode para convertir las listas en filas individuales
games = games.explode('tags')
# Mostrar los valores únicos

In [27]:
tags_genres = [
    "Action",
    "Casual",
    "Indie",
    "Simulation",
    "Strategy",
    "Free to Play",
    "RPG",
    "Sports",
    "Adventure",
    "Racing",
    "Early Access",
    "Massively Multiplayer",
    "Animation & Modeling",
    "Video Production",
    "Utilities",
    "Web Publishing",
    "Education",
    "Software Training",
    "Design & Illustration",
    "Audio Production",
    "Photo Editing",
    "Accounting"
]

# Conservar solo las columnas en tags_drop y eliminar el resto
games = games[games['tags'].isin(tags_genres)]


In [28]:
games['genres'] = games['genres'].fillna(games['tags'])

In [29]:
games = games.explode('genres')

In [30]:
games = games.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
games.columns

Index(['genres', 'price', 'early_access', 'id', 'release_year', 'publisher',
       'app_name', 'title', 'developer'],
      dtype='object')

In [31]:
games = games.drop_duplicates()

In [36]:
games.isna().sum().sort_values(ascending= False)/len(games) * 100

genres          0.0
price           0.0
early_access    0.0
id              0.0
release_year    0.0
publisher       0.0
app_name        0.0
title           0.0
developer       0.0
dtype: float64

In [33]:

# Modifica todos los registros que cumplen con la condición en la columna especificada
games.loc[(games['genres'] == 'Animation &amp; Modeling'), 'genres'] = 'Animation & Modeling'


In [34]:
games.loc[(games['genres'] == 'Design &amp; Illustration'), 'genres'] = 'Design & Illustration'

In [35]:
games = games.drop_duplicates()

In [38]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77930 entries, 0 to 32134
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        77930 non-null  object 
 1   price         77930 non-null  float64
 2   early_access  77930 non-null  object 
 3   id            77930 non-null  object 
 4   release_year  77930 non-null  object 
 5   publisher     77930 non-null  object 
 6   app_name      77930 non-null  object 
 7   title         77930 non-null  object 
 8   developer     77930 non-null  object 
dtypes: float64(1), object(8)
memory usage: 8.0+ MB
